# 🏏 ICC World Cup Finals: Cricket Highlights Monitoring using VideoDB RTStream
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/real_time_streaming/Cricket_Match_Monitoring.ipynb)
## 📖 Storytime: Why This Matters

Tonight is the **ICC World Cup finals between India and Pakistan**.  
Millions are watching the match live, and the competition to post match highlights — sixes, wickets, and spectacular catches — on social media is fiercer than ever.  

In the usual workflow, someone watches the match, waits for a moment to happen, then clips the video manually and uploads it online — often **several minutes too late**.

But we have a smarter way.

What if AI could monitor the match for you, detect key moments in real-time, and instantly send alerts when something exciting happens — giving you a headstart on posting highlights while everyone else scrambles?  

**Let’s build exactly that using VideoDB RTStream.**

---

## 🚀 What You’ll Build in This Notebook

By the end of this notebook, you'll learn how to:
- Connect a cricket match video stream to VideoDB
- Use AI to continuously analyze match footage and describe key moments
- Detect real-time events like **sixes**, **fours**, **catches**, and **wickets**
- Trigger instant alerts for each event through webhooks

Let’s get started!

---

## 📦 Step 1: Install Dependencies

We’ll begin by installing the VideoDB SDK

In [1]:
!pip install -q videodb

  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Let's connect to VideoDB's API using your credentials to prepare for stream monitoring.

Please enter your `VIDEO_DB_API_KEY` in the input box that appears below after you run this cell.

Your input will be masked.


In [2]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---

## 📦 Step 3: Connect to the Cricket Match RTSP Stream

We’ll now connect to the live video stream of the match using its RTSP URL.
In this demo, the stream is running at `rtsp://samples.rts.videodb.io:8554/cricket`.


In [ ]:
rtsp_url = "rtsp://samples.rts.videodb.io:8554/cricket"
cricket_stream = coll.connect_rtstream(
    name="Cricket Finals Stream",
    url=rtsp_url,
)
print(cricket_stream)

RTStream(id=rts-019711db-1086-7750-ba79-8f47a4fed603, name=Cricket Finals Stream, collection_id=None, created_at=None, sample_rate=30, status=connected)


#### Let us list all the rtstreams in our collection.

In [ ]:
def list_rtstreams():
    for rtstream in coll.list_rtstreams():
        print(f"""RTStream:
        ID            : {rtstream.id}
        Name          : {rtstream.name}
        Collection ID : {rtstream.collection_id}
        Created At    : {rtstream.created_at}
        Sample Rate   : {rtstream.sample_rate}
        Status        : {rtstream.status}
        """)
        print("-" * 80)

list_rtstreams()

RTStream:
        ID            : rts-019711db-1086-7750-ba79-8f47a4fed603
        Name          : Cricket Finals Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T13:07:38.758741
        Sample Rate   : 30
        Status        : connected
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019711a0-0fde-7911-b282-25bc0b4ecf65
        Name          : Baby Crib Monitor
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T12:03:11.966358
        Sample Rate   : 30
        Status        : stopped
        
--------------------------------------------------------------------------------
RTStream:
        ID            : rts-019710fa-9511-79c3-a924-e229e4815410
        Name          : Property Security Stream
        Collection ID : c-81fc6459-fe30-44ac-8c5b-ea0898c2e152
        Created At    : 2025-05-27T09:02:27.089


#### If you have already connected the stream, run the below cell with the **rtstream id** to reconnect.

In [3]:
# cricket_stream = coll.get_rtstream("")

In [ ]:
# To stop the stream
# cricket_stream.stop()

In [4]:
# To start the stream
# cricket_stream.start()

---
### 👀 Let us have a look at the cricket match

#### 📺 Helper Functions: Search and Display

This cell contains all the utility functions to search, fetch, and visualize video streams. You don't need to modify this code.

In [ ]:
# To display the stream with relevant information

from IPython.display import HTML
import re
import time
from datetime import datetime, UTC
from videodb import play_stream


def display_stream(video_url, video_name="🎥 Camera Feed"):
    match = re.search(r"/(\d{16})-(\d{16})\.m3u8", video_url)
    if match:
        start_ts = int(match.group(1)) / 1e6
        end_ts = int(match.group(2)) / 1e6
        start_time = datetime.fromtimestamp(start_ts, UTC).strftime("%Y-%m-%d %H:%M:%S")
        end_time = datetime.fromtimestamp(end_ts, UTC).strftime("%Y-%m-%d %H:%M:%S")
        time_range = f"{start_time} → {end_time} UTC"
    else:
        time_range = "Time Unknown"

    video_player_html = play_stream(video_url)

    return HTML(f"""
    <div style="position:relative;width:640px;">
      {video_player_html._repr_html_() if hasattr(video_player_html, "_repr_html_") else video_player_html}
      <div style="position:absolute;top:10px;left:10px;background:rgba(0,0,0,0.6);color:#fff;padding:6px 12px;border-radius:4px;font-size:13px;font-family:sans-serif;">
        <strong>{video_name}</strong><br>{time_range}
      </div>
    </div>
    """)


# To dynamically set the display duration


def prompt_to_time(prompt):
    now = int(time.time())
    prompt = (
        f"It's {now} in epoch seconds. "
        f"Convert the phrase '{prompt}' into JSON "
        f'with keys "from" and "to" (both epoch seconds)'
    )

    result = coll.generate_text(
        prompt=prompt,
        model_name="pro",
        response_type="json",
    )
    output = result.get("output", {})
    return output.get("from"), output.get("to")


# To fetch stream


def fetch_stream(rtstream):
    _from, to = prompt_to_time("Show me last 2 mins")
    stream_url = rtstream.generate_stream(_from, to)
    return stream_url



#### 🔗 Get & Display Recent Stream

This cell uses the helper functions above to fetch and display the last few minutes of the stream.

In [ ]:
# To get last few minutes stream link

video_url = fetch_stream(cricket_stream)

video_name = "🏏 ICC World Cup · India vs Pakistan"
display_stream(video_url, video_name)

---
## 📦 Step 4: Index Scenes and Describe Match Highlights

We’ll create a real-time scene index that periodically analyzes the video and generates natural language descriptions of what’s happening in the match.

The AI model will look out for:
- Sixes (ball hit over the boundary rope without bouncing)
- Fours (ball crosses boundary rope after bouncing)
- Catches (fielder catches the ball before it touches the ground)
- Wickets (batsman is dismissed)

In [ ]:
from videodb import SceneExtractionType

cricket_scene_index = cricket_stream.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={
        "time": 7,
        "frame_count": 7,
    },
    prompt="Identify and mention when a batsman hits a SIX (ball flying over the boundary rope), a FOUR (ball crosses boundary rope after bouncing), a CATCH OUT (fielder catches the ball mid-air before it touches ground) or a WICKET (when the wicket stumps are put down by the ball).",
    name="Cricket_Match_Scene_Index",
)
cricket_index_id = cricket_scene_index.rtstream_index_id
print("Scene Index ID:", cricket_index_id)

Scene Index ID: c251098cf8fc4561


#### Let us list the scene indexes created on our rtstream.

In [ ]:
def list_rtstream_indexes(rtstream):
    # List live stream indexes
    rtstream_indexes = rtstream.list_scene_indexes()
    for rtstream_index in rtstream_indexes:

        print(f"""RTStreamSceneIndex:
            Index ID       : {rtstream_index.rtstream_index_id}
            RTStream ID    : {rtstream_index.rtstream_id}
            Name           : {rtstream_index.name}
            Status         : {rtstream_index.status}
            Config         : {rtstream_index.extraction_config}
            Prompt         : {rtstream_index.prompt}
        """)
        print("-" * 80)

list_rtstream_indexes(cricket_stream)

RTStreamSceneIndex:
            Index ID       : c251098cf8fc4561
            RTStream ID    : rts-019711db-1086-7750-ba79-8f47a4fed603
            Name           : Cricket_Match_Scene_Index
            Status         : running
            Config         : {'frame_count': '7', 'time': '7'}
            Prompt         : Identify and mention when a batsman hits a SIX (ball flying over the boundary rope), a FOUR (ball crosses boundary rope after bouncing), a CATCH OUT (fielder catches the ball mid-air before it touches ground) or a WICKET (when the wicket stumps are put down by the ball).
        
--------------------------------------------------------------------------------
RTStreamSceneIndex:
            Index ID       : cd031ae88e755900
            RTStream ID    : rts-019711db-1086-7750-ba79-8f47a4fed603
            Name           : Cricket_Scene_Index
            Status         : stopped
            Config         : {'frame_count': '7', 'time': '7'}
            Prompt         : Moni


#### If you have already created a scene index, run the below cell with your **scene index id** to reconnect.

In [ ]:
# cricket_index_id = ""
# cricket_scene_index = cricket_stream.get_scene_index(cricket_index_id)

In [ ]:
# To stop the index
# cricket_scene_index.stop()

In [ ]:
# To start the index
# cricket_scene_index.start()

---
### Let us see the result of the scene indexing

In [ ]:
import time
from datetime import datetime
from zoneinfo import ZoneInfo

def _convert_to_ist(timestamp: float) -> str:
    """Convert UTC timestamp to IST (Asia/Kolkata) datetime string."""
    return (
        datetime.fromtimestamp(timestamp)
        .astimezone(ZoneInfo("Asia/Kolkata"))
        .strftime("%Y-%m-%d %H:%M:%S")
    )

def get_scenes(rtstream, index_id):
    # Print indexed scenes
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    scenes = rtstream_scene_index.get_scenes(page_size=5)
    if scenes:
        for scene in scenes.get("scenes"):
            start = _convert_to_ist(scene["start"])
            end = _convert_to_ist(scene["end"])
            description = scene["description"]
            print(f"{start}-{end}: {description}")
            print("-" * 80)
    else:
        print("Scenes not found for given index.")

get_scenes(cricket_stream , cricket_index_id)

2025-05-27 19:00:38-2025-05-27 19:00:45: Here's what I can identify from the images:

*   **Image 1:** The umpire is signaling a FOUR.
*   **Image 7:** The ball is hitting the wickets. This indicates a WICKET.
--------------------------------------------------------------------------------
2025-05-27 19:00:30-2025-05-27 19:00:37: Okay, I can identify those events in the images you provided.

*   **Image 6:** WICKET - The batsman is out as the ball has hit the stumps.
*   **Image 7:** WICKET - The umpire signals that the batsman is out.
--------------------------------------------------------------------------------
2025-05-27 19:00:21-2025-05-27 19:00:29: Based on the information available in the images, I can identify the following:

*   **FOUR:** At 0:02 in the video, the scorecard shows a "4" for the current ball. This indicates that the batsman hit a FOUR.
--------------------------------------------------------------------------------
2025-05-27 19:00:13-2025-05-27 19:00:20: Here'

## 📦 Step 5: Define Events for Each Match Highlight

Now, let’s define four events to detect each key match moment.


1. Sixes (ball hit over the boundary rope without bouncing)

In [ ]:
six_event_id = conn.create_event(
    event_prompt="Detect when a batsman hits a SIX.",
    label="six_hit"
)

2. Fours (ball crosses boundary rope after bouncing)


In [ ]:
four_event_id = conn.create_event(
    event_prompt="Detect when a batsman hits a FOUR.",
    label="four_hit"
)

3. Catches (fielder catches the ball before it touches the ground)


In [ ]:
catch_event_id = conn.create_event(
    event_prompt="Detect when a player takes a CATCH OUT.",
    label="catch_out"
)

4. Wickets (batsman is dismissed)

In [ ]:
wicket_event_id = conn.create_event(
    event_prompt="Detect when a batsman is dismissed (WICKET).",
    label="wicket"
)

Let us have a look at all the events created:

In [ ]:
print(f"Successfully created events with the following IDs:\n"
      f"- Six Event ID: {six_event_id}\n"
      f"- Four Event ID: {four_event_id}\n"
      f"- Catch Event ID: {catch_event_id}\n"
      f"- Wicket Event ID: {wicket_event_id}")

Successfully created events with the following IDs:
- Six Event ID: 627cdd8124f7731b
- Four Event ID: 3bfdd25d9239861b
- Catch Event ID: 16b35b66ef106a37
- Wicket Event ID: fae540ba13f50530


---

## 📦 Step 6: Attach Alerts for Each Event

We’ll create four different alerts — one for each event — but route them all to the same Pipedream webhook.


In [ ]:
# Enter link to your webhook url where you want alerts to go. You can create one simply on pipedream.
webhook_url=""

1. Create an alert for 'six' events and store its ID

In [ ]:
if webhook_url:
  six_alert_id = cricket_scene_index.create_alert(
      event_id=six_event_id,
      callback_url=webhook_url
  )
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

2. Create an alert for 'four' events and store its ID

In [ ]:
if webhook_url:
  four_alert_id = cricket_scene_index.create_alert(
      event_id=four_event_id,
      callback_url=webhook_url
  )
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

3. Create an alert for 'catch' events and store its ID

In [ ]:
if webhook_url:
  catch_alert_id = cricket_scene_index.create_alert(
      event_id=catch_event_id,
      callback_url=webhook_url
  )
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

4. Create an alert for 'wicket' events and store its ID

In [ ]:
if webhook_url:
  wicket_alert_id = cricket_scene_index.create_alert(
      event_id=wicket_event_id,
      callback_url=webhook_url
  )
else:
  print("Error: Please provide Webhook URL. Alert cannot be created without it.")

---
#### Lets list all the alerts

In [ ]:
def list_rtstream_alerts(rtstream, index_id):
    """
    Prints a list of alerts associated with a given scene index.
    """
    rtstream_scene_index = rtstream.get_scene_index(index_id)
    alerts = rtstream_scene_index.list_alerts()

    for alert in alerts:
        print(f"""🔔 RTStream Alert:
    Alert ID      : {alert['alert_id']}
    Event ID      : {alert['event_id']}
    Label         : {alert['label']}
    Prompt        : {alert['prompt']}
    Status        : {alert['status']}
        """)
        print("-" * 80)

list_rtstream_alerts(cricket_stream, cricket_index_id)


🔔 RTStream Alert:
    Alert ID      : 1891843a2c669ea3
    Event ID      : 3bfdd25d9239861b
    Label         : four_hit
    Prompt        : Detect when a batsman hits a FOUR.
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : 6094972d46c861be
    Event ID      : 16b35b66ef106a37
    Label         : catch_out
    Prompt        : Detect when a player takes a CATCH OUT.
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : 69d79872465ce9f2
    Event ID      : 627cdd8124f7731b
    Label         : six_hit
    Prompt        : Detect when a batsman hits a SIX.
    Status        : enabled
        
--------------------------------------------------------------------------------
🔔 RTStream Alert:
    Alert ID      : b4ae6c64ea065ca8
    Event ID      : fae540ba13f50530
    Label         : wi

---

## 📡 Example Alerts Received at the Webhook

1. Alert received when a FOUR is hit

```json
{
  "event_id": "event-3bfdd25d9239861b",
  "label": "four_hit",
  "confidence": 0.95,
  "explanation": "Image 4 shows the ball crossing the boundary after bouncing, indicating a FOUR has been scored. Therefore, an alert should be triggered.",
  "timestamp": "2025-05-29T00:11:09.256447+00:00",
  "start_time": "2025-05-29T05:40:32.544547+05:30",
  "end_time": "2025-05-29T05:40:39.730362+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019711db-1086-7750-ba79-8f47a4fed603/1748477432000000-1748477440000000.m3u8"
}
```

2. Alert recieved on a CATCH OUT

```json
{
  "event_id": "event-16b35b66ef106a37",
  "label": "catch_out",
  "confidence": 0.95,
  "explanation": "Image 4 shows a fielder catching the ball mid-air, indicating a CATCH OUT, which aligns with the alert context.",
  "timestamp": "2025-05-29T00:10:52.331198+00:00",
  "start_time": "2025-05-29T05:40:24.326385+05:30",
  "end_time": "2025-05-29T05:40:31.314371+05:30",
  "stream_url": "https://rt.stream.videodb.io/manifests/rts-019711db-1086-7750-ba79-8f47a4fed603/1748477424000000-1748477432000000.m3u8"
}
```

✅ Similarly, alerts will be triggered for **SIX** and **WICKET** events, routed to the same webhook URL.

---
### Let us have a look at the stream links we received in the alerts.

1. FOUR HIT

In [7]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-019711db-1086-7750-ba79-8f47a4fed603/1748477432000000-1748477440000000.m3u8"
video_name = "🏏 ICC World Cup · four_hit"

display_stream(alert_stream_url,video_name)

2. CATCH OUT

In [8]:
alert_stream_url = "https://rt.stream.videodb.io/manifests/rts-019711db-1086-7750-ba79-8f47a4fed603/1748477424000000-1748477432000000.m3u8"
video_name = "🏏 ICC World Cup · catch_out"

display_stream(alert_stream_url,video_name)

---
- After the match is over, we can disable the alerts.

In [ ]:
cricket_scene_index.disable_alert(six_alert_id)
cricket_scene_index.disable_alert(four_alert_id)
cricket_scene_index.disable_alert(catch_alert_id)
cricket_scene_index.disable_alert(wicket_alert_id)

- To enable the alerts again:

In [ ]:
cricket_scene_index.enable_alert(six_alert_id)
cricket_scene_index.enable_alert(four_alert_id)
cricket_scene_index.enable_alert(catch_alert_id)
cricket_scene_index.enable_alert(wicket_alert_id)

- Now we can stop the stream

In [ ]:
cricket_stream.stop()

---
## 🏆 Wrapping Up: Outpacing the Competition in Real Time

In this notebook, we built a smart, AI-powered video monitoring system tailored for the high-pressure, real-time demands of live sports broadcasting.

With this setup in place, broadcasters and content creators no longer have to wait, clip, and scramble.  
They can stay ahead of the crowd, instantly catching and sharing match-defining moments as they happen — turning every six, four, wicket, and catch into social media gold within seconds.

---

**But this is just one story.**

- What if the same system could monitor a **football match**, detecting goals, penalties, and crowd invasions in real time?  
- Or watch over an **esports championship**, capturing game-winning moves and viral moments?  
- Or even scan **live news broadcasts**, flagging breaking events or significant headlines for immediate clipping and distribution?  

The possibilities of real-time video intelligence in media and sports are endless.

**What would *you* stream next?**
